In [2]:
# import the necessary libraries
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import pandas as pd
import imutils
import time
import dlib
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def eye_aspect_ratio(eye):
    # Vertical
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])  
    # Horizontal
    C = dist.euclidean(eye[0], eye[3])
    # EAR
    ear = (A + B)/(2.0 * C)
    
    return ear

In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [5]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [54]:
# EYE THRESHOLD DETERMINATION

thresh = pd.DataFrame()
dir = r'C:\Users\pnanp\Desktop\Minor.AI4S.S6\Personal Project\Data\Fold3_part1'

# Looping through directory for mp4 files to read
for file in os.listdir(dir):
    if file.endswith(".mp4"):
        path=os.path.join(dir, file)
        print('Initiating {}'.format(file))

        
        # Defining lists to store attributes
        allEars = []
        avgEars = []

        # Read the video
        vs = FileVideoStream(path)
        vs.start()
        time.sleep(1.0)
        
        # While frame is incoming, read the frame
        while vs.more() == True:  
            frame = vs.read()
            # If the frame is not of type None, perform adjustments and face detection
            if vs.more() == True:
                frame =  imutils.resize(frame, width=450)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frameCount += 1     # Count and track each frame

                rects = detector(gray, 0)
                
                for rect in rects:
                    # Predict the landmarks and convert their coordinates into an array
                    shape =  predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    # Extract the coordinates of left and right eyesusing the indexes derived before
                    leftEye = shape[lStart:lEnd]
                    rightEye = shape[rStart:rEnd]
                    # Calculate EARs and take average
                    leftEAR = eye_aspect_ratio(leftEye)
                    rightEAR = eye_aspect_ratio(rightEye)

                    ear = (leftEAR + rightEAR) / 2.0

                    allEars.append(ear)     # List to track all EARs
                    
        avgEarCal = sum(allEars)/len(allEars)
        avgEars.append(avgEarCal)
        print(avgEars)
                    
        print('{} features extracted'.format(file))
        vs.stop()
        cv2.destroyAllWindows()
        
        # Creating and extending dataframe
        mediumDft = pd.DataFrame({'avgEAR': avgEars,'State': file})
        thresh = thresh.append(mediumDft, ignore_index = True)
        print('Data updated to {} rows'.format(len(thresh)))
        print('----------------------------------------')

Initiating 0s26.mp4
[0.3723883208862469]
0s26.mp4 features extracted
Data updated to 1 rows
----------------------------------------
Initiating 0s27.mp4
[0.2958811250463722]
0s27.mp4 features extracted
Data updated to 2 rows
----------------------------------------
Initiating 0s28.mp4
[0.24334681255806584]
0s28.mp4 features extracted
Data updated to 3 rows
----------------------------------------
Initiating 0s29.mp4
[0.24450091695457882]
0s29.mp4 features extracted
Data updated to 4 rows
----------------------------------------
Initiating 0s30.mp4
[0.3272312940802247]
0s30.mp4 features extracted
Data updated to 5 rows
----------------------------------------
Initiating 10s26.mp4
[0.3295268225649787]
10s26.mp4 features extracted
Data updated to 6 rows
----------------------------------------
Initiating 10s27.mp4
[0.28052664425904333]
10s27.mp4 features extracted
Data updated to 7 rows
----------------------------------------
Initiating 10s28.mp4
[0.2666157124465833]
10s28.mp4 features e

In [55]:
thresh.to_csv('threshf3p1.csv')